In [1]:
import os
import glob
import pandas as pd

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/SMPD3/TCFLefH2Bd2EGFPIntensity'

In [5]:
# Add experiment date here to apply to dataframe
analysis_date = '20190401'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe


for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label
    df['Fluor'], df['ROI'] = zip(*df['Label'].map(lambda x: x.split(':')))
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Stains'], df['Embryo'], 
        df['Somites'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    
    # Replace background ROI names
        # NOTE: I have updated the Fiji macro ('FluorIntensity_2Channel.ijm') to name all background ROIs as 'background',
        # so this step will be unnecessary with freshly collected data
    #df.replace(to_replace=['back1a', 'back1b', 'back1c', 'back2a', 'back2b', 'back2c'],
     #                       value=['background', 'background', 'background', 'background', 'background', 'background'], 
      #                      inplace=True)
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,IntDen,RawIntDen,Image,Fluor,ROI,ExptDate,Treatment,Stains,Embryo,Somites,Section
0,1,H2BRFP:background,34.937,8.134,284.182,5515.0,"20190327_SMPD3MO0,8mM_Pax7RFPTCFLefd2eGFPDAPI_...",H2BRFP,background,20190327,"SMPD3MO0,8mM",Pax7RFPTCFLefd2eGFPDAPI,Emb7,7ss,sec2
1,2,H2BRFP:background,36.792,8.443,310.617,6028.0,"20190327_SMPD3MO0,8mM_Pax7RFPTCFLefd2eGFPDAPI_...",H2BRFP,background,20190327,"SMPD3MO0,8mM",Pax7RFPTCFLefd2eGFPDAPI,Emb7,7ss,sec2
2,3,H2BRFP:background,44.830,8.178,366.629,7115.0,"20190327_SMPD3MO0,8mM_Pax7RFPTCFLefd2eGFPDAPI_...",H2BRFP,background,20190327,"SMPD3MO0,8mM",Pax7RFPTCFLefd2eGFPDAPI,Emb7,7ss,sec2
3,4,H2BRFP:Cntl,3835.716,347.719,1333749.738,25883478.0,"20190327_SMPD3MO0,8mM_Pax7RFPTCFLefd2eGFPDAPI_...",H2BRFP,Cntl,20190327,"SMPD3MO0,8mM",Pax7RFPTCFLefd2eGFPDAPI,Emb7,7ss,sec2
4,5,H2BRFP:Expt,2291.907,638.603,1463618.689,28403786.0,"20190327_SMPD3MO0,8mM_Pax7RFPTCFLefd2eGFPDAPI_...",H2BRFP,Expt,20190327,"SMPD3MO0,8mM",Pax7RFPTCFLefd2eGFPDAPI,Emb7,7ss,sec2


In [6]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean background values and group by Treatment, Embryo, Fluor, ROI and Section
mean_sections = ((full_df.groupby(['Treatment', 'Embryo', 'Fluor', 'ROI', 'Section', 'ExptDate'])
                  ['Area', 'Mean', 'IntDen']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))

    # Determine CTCF values = ROI IntDen - (background mean * ROI area)
    # Calculate background (background mean * ROI area)
    background_corr_cntl = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Cntl', level='ROI')['Area'])
    background_corr_expt = (df_treatment.xs('background', level='ROI')['Mean'] 
                    * df_treatment.xs('Expt', level='ROI')['Area'])

    # Slice out only Cntl or Expt values in IntDen
    intdens_cntl = df_treatment.xs('Cntl', level='ROI')['IntDen'] 
    intdens_expt = df_treatment.xs('Expt', level='ROI')['IntDen'] 

    # Subtract background from IntDens to determine CTCF and concatenate into single dataframe
    sub_cntl = pd.DataFrame(intdens_cntl - background_corr_cntl)
    sub_expt = pd.DataFrame(intdens_expt - background_corr_expt)
    full_ctcf = pd.concat([sub_cntl, sub_expt], keys = ['Cntl', 'Expt'])
    full_ctcf.columns = ['CTCF']

    # Pull out TCFLef and H2BRFP values
    ctcf_tcflef = full_ctcf.xs('TCFLefd2eGFP', level='Fluor')['CTCF'] 
    ctcf_RFP = full_ctcf.xs('H2BRFP', level='Fluor')['CTCF'] 

    # Normalize for electroporation efficiency by determining TCFLef/RFP
    electroporation_norm = pd.DataFrame(ctcf_tcflef / ctcf_RFP)
    electroporation_norm.columns = ['CTCF']
    electroporation_norm.index.names = ['Side', 'Embryo', 'Section', 'ExptDate']

    # Average sections grouped by embryos before generating Expt/Cntl ratio
    averaged_sections = electroporation_norm.groupby(['Side','Embryo', 'ExptDate']).mean()

    # Pull out Cntl and Expt CTCFs
    ctcf_cntl = averaged_sections.xs('Cntl', level='Side')['CTCF'] 
    ctcf_expt = averaged_sections.xs('Expt', level='Side')['CTCF'] 

    # Generate ratios as Expt/Cntl
    ratios_sections = pd.DataFrame(ctcf_expt / ctcf_cntl)
    ratios_sections.columns = ['Expt/Cntl CTCF']

    # Normalize individual values to mean of control group
    norm_cntl = ctcf_cntl/(float(ctcf_cntl.mean()))
    norm_cntl = pd.DataFrame(norm_cntl)
    norm_cntl.columns = ['Norm Cntl CTCF']
    norm_expt = ctcf_expt/(float(ctcf_cntl.mean()))
    norm_expt = pd.DataFrame(norm_expt)
    norm_expt.columns = ['Norm Expt CTCF']

    # Combine processed values into single dataframe and output as csv file 'Results.csv'
    ctcf_cntl = pd.DataFrame(ctcf_cntl)
    ctcf_cntl.columns = ['Cntl CTCF']
    ctcf_expt = pd.DataFrame(ctcf_expt)
    ctcf_expt.columns = ['Expt CTCF']
    results = (pd.concat([ctcf_cntl, ctcf_expt, ratios_sections, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
    results['ID'] = results.ExptDate.str.cat(results.Embryo)
    results.to_csv(analysis_date + '_' + treatment + '_CTCFResults.csv')